In [ ]:
# Note jupyter notebook cant seem to open the viewing window

In [2]:
#Libraries
from pyueye import ueye
import numpy as np
# import cv2
import sys
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#Variables
hCam = ueye.HIDS(0)             #0: first available camera;  1-254: The camera with the specified camera ID
sInfo = ueye.SENSORINFO()
cInfo = ueye.CAMINFO()
pcImageMemory = ueye.c_mem_p()
MemID = ueye.int()
rectAOI = ueye.IS_RECT()
pitch = ueye.INT()
nBitsPerPixel = ueye.INT(8)    #24: bits per pixel for color mode; take 8 bits per pixel for monochrome
channels = 1                    #3: channels for color mode(RGB); take 1 channel for monochrome
m_nColorMode = ueye.INT()		# Y8/RGB16/RGB24/REG32
bytes_per_pixel = int(nBitsPerPixel / 8)
#---------------------------------------------------------------------------------------------------------------------------------------
print("START")
print()


START



In [4]:
# Starts the driver and establishes the connection to the camera
nRet = ueye.is_InitCamera(hCam, None)
if nRet != ueye.IS_SUCCESS:
    print("is_InitCamera ERROR")



In [5]:
# Can be used to set the size and position of an "area of interest"(AOI) within an image
nRet = ueye.is_AOI(hCam, ueye.IS_AOI_IMAGE_GET_AOI, rectAOI, ueye.sizeof(rectAOI))
if nRet != ueye.IS_SUCCESS:
    print("is_AOI ERROR")
width = rectAOI.s32Width
height = rectAOI.s32Height

In [6]:
# Prints out some information about the camera and the sensor
print("Camera model:\t\t", sInfo.strSensorName.decode('utf-8'))
print("Camera serial no.:\t", cInfo.SerNo.decode('utf-8'))
print("Maximum image width:\t", width)
print("Maximum image height:\t", height)
print()

Camera model:		 
Camera serial no.:	 
Maximum image width:	 3840
Maximum image height:	 2748



In [7]:
exposure_time = 5.0
exposure_time_ue = ueye.c_double(exposure_time)
nRet = ueye.is_Exposure(hCam, ueye.IS_EXPOSURE_CMD_SET_EXPOSURE, exposure_time_ue, 8)

In [8]:
print(ueye.double(ueye.IS_EXPOSURE_CMD_GET_EXPOSURE))

print(ueye.double(ueye.IS_EXPOSURE_CMD_GET_EXPOSURE_RANGE_MIN))
print(ueye.double(ueye.IS_EXPOSURE_CMD_GET_EXPOSURE_RANGE_MAX))

7.0
3.0
4.0


In [12]:
help(ueye.is_Exposure)

Help on function is_Exposure in module pyueye.ueye:

is_Exposure(hCam, nCommand, pParam, cbSizeOfParam)
    :param hCam: c_uint (aka c-type: HIDS)
    :param nCommand: c_uint (aka c-type: UINT)
    :param pParam: c_void_p - might differ depending on nCommand (aka c-type: void \*)
    :param cbSizeOfParam: c_uint (aka c-type: UINT)
    :returns: success, or no success, that is the answer
    :raises NotImplementedError: if function could not be loaded



In [9]:
itime_get_p = ueye.c_double(5)

ueye.is_Exposure(hCam, ueye.IS_EXPOSURE_CMD_GET_EXPOSURE, itime_get_p, 8)

itime_get_p.value

5.047583333333334

In [28]:
itime_get_p = ueye.c_double(666)

ueye.is_Exposure(hCam, ueye.IS_EXPOSURE_CMD_GET_EXPOSURE_RANGE_MAX, itime_get_p, 8)

itime_get_p.value

499.92283333333336

In [10]:

# Allocates an image memory for an image having its dimensions defined by width and height and its color depth defined by nBitsPerPixel
nRet = ueye.is_AllocImageMem(hCam, width, height, nBitsPerPixel, pcImageMemory, MemID)
if nRet != ueye.IS_SUCCESS:
    print("is_AllocImageMem ERROR")
else:
    # Makes the specified image memory the active memory
    nRet = ueye.is_SetImageMem(hCam, pcImageMemory, MemID)
    if nRet != ueye.IS_SUCCESS:
        print("is_SetImageMem ERROR")
    else:
        # Set the desired color mode
        nRet = ueye.is_SetColorMode(hCam, m_nColorMode)


In [11]:
# Activates the camera's live video mode (free run mode)
nRet = ueye.is_SetDisplayMode(hCam, ueye.IS_DONT_WAIT)
if nRet != ueye.IS_SUCCESS:
    print("is_CaptureVideo ERROR")

In [8]:
# Activates the camera's live video mode (free run mode)
nRet = ueye.is_CaptureVideo(hCam, ueye.IS_DONT_WAIT)
if nRet != ueye.IS_SUCCESS:
    print("is_CaptureVideo ERROR")

In [9]:
nRet = ueye.is_SetFrameRate(hCam, 1000, ueye.double(100))
if nRet != ueye.IS_SUCCESS:
    print("is_SetFrameRate ERROR")


In [10]:
# Enables the queue mode for existing image memory sequences
nRet = ueye.is_InquireImageMem(hCam, pcImageMemory, MemID, width, height, nBitsPerPixel, pitch)
if nRet != ueye.IS_SUCCESS:
    print("is_InquireImageMem ERROR")
else:
    print("Press q to leave the programm")

Press q to leave the programm


In [13]:
array = ueye.get_data(pcImageMemory, width, height, nBitsPerPixel, pitch, copy=False)

print(array)
frame = np.reshape(array,(height.value, width.value, bytes_per_pixel))
plt.imshow(np.abs(frame))
plt.colorbar()

[]


ValueError: cannot reshape array of size 0 into shape (2748,3840,1)

In [15]:
# Continuous image display
while(nRet == ueye.IS_SUCCESS):

    # In order to display the image in an OpenCV window we need to...
    # ...extract the data of our image memory
    array = ueye.get_data(pcImageMemory, width, height, nBitsPerPixel, pitch, copy=False)

    # bytes_per_pixel = int(nBitsPerPixel / 8)

    # ...reshape it in an numpy array...
    frame = np.reshape(array,(height.value, width.value, bytes_per_pixel))

    # ...resize the image by a half
    frame = cv2.resize(frame,(0,0),fx=0.5, fy=0.5)

KeyboardInterrupt: 

In [14]:
# Releases an image memory that was allocated using is_AllocImageMem() and removes it from the driver management
ueye.is_FreeImageMem(hCam, pcImageMemory, MemID)

# Disables the hCam camera handle and releases the data structures and memory areas taken up by the uEye camera
ueye.is_ExitCamera(hCam)

# Destroys the OpenCv windows
# cv2.destroyAllWindows()

0

In [15]:
import numpy as np

In [19]:
wr = np.arange(600,700, 10)
bw = 5
sw = wr - bw/2
lw = wr + bw/2

print(sw, '\n', lw)

[597.5 607.5 617.5 627.5 637.5 647.5 657.5 667.5 677.5 687.5] 
 [602.5 612.5 622.5 632.5 642.5 652.5 662.5 672.5 682.5 692.5]


In [20]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2024-02-06'

In [24]:
a = np.array([1,2,3,5,4,3,2])
np.where(a == np.max(a))[0][0]

3